<a href="https://colab.research.google.com/github/juliocesarortiz/DataScience/blob/main/DSA05_STREAMLIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# instalar streamlit
!pip install -q streamlit

# bajar ngrok
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar xvf ngrok-v3-stable-linux-amd64.tgz

# guardar token
!./ngrok config add-authtoken 2izjNE46GsCckEbRwE4ndLIPRLn_J7wx98qcjDWxxLXsKNez

# Levantar tunel
!./ngrok http 8501 &>/content/ngrok.log &

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.3 MB/s eta 0:00:00
--2024-07-15 03:30:35--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8967830 (8.6M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz’

ngrok-v3-stable-lin 100%[===================>]   8.55M  30.0MB/s    in 0.3s    

2024-07-15 03:30:35 (30.0 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz’ saved [8967830/8967830]

ngrok
Authtoken

In [62]:
%%writefile app.py

import pandas as pd
import streamlit as st
import plotly.express as px

st.set_page_config(layout="wide")

@st.cache_data
def load_data(num=500):
  data = pd.read_csv("/content/Employees.csv",nrows=num)
  return data

data = load_data() # Moved outside the function to make it accessible
#show_data = st.sidebar.checkbox("Mostrar datos")
 # Crear título de la aplicación
st.title("Dashboard de empleados")
# Crear encabezados
st.header("Datos de empleados")
st.subheader("Descripción:")
# Crear texto de la descripción del proyecto
st.write(
    """
    Este dashboard muestra información sobre los empleados de una empresa.
    Los datos se cargan desde un archivo CSV y se muestran en una tabla.
    El usuario puede seleccionar el número de filas que desea cargar.
    """
)
show_original_data = st.sidebar.checkbox("Mostrar datos originales")
# Crear buscador de empleados por employee id
employee_id = st.sidebar.text_input("Buscar empleado por ID")


# Filtrar datos por employee id
if employee_id:
   # Búsqueda parcial
  data = data[data["Employee_ID"].str.contains(employee_id, case=False, regex=False)]
  if data.empty:
    st.write("No se encontró ningún empleado con ese ID")

# Crear un control de selección para filtrar por nivel educativo
education_level = st.sidebar.selectbox(
    "Filtrar por nivel educativo",
    options=data["Education_Level"].unique()
)
# Filtrar los datos por el nivel educativo seleccionado
if education_level:
    data = data[data["Education_Level"] == education_level]
# Crear un control de selección para filtrar por ciudad
city = st.sidebar.selectbox(
    "Filtrar por ciudad",
    options=data["Hometown"].unique()
)

# Filtrar los datos por la ciudad seleccionada
if city:
    data = data[data["Hometown"] == city]
# Crear un control de selección para filtrar por unidad funcional
unit = st.sidebar.selectbox(
    "Filtrar por unidad funcional",
    options=data["Unit"].unique()
)

# Filtrar los datos por la unidad funcional seleccionada
if unit:
    data = data[data["Unit"] == unit]
# Mostrar el total de empleados que cumplen con los filtros aplicados
st.write("Total de empleados:", len(data))
# Mostrar los datos filtrados
st.dataframe(data)

# Mostrar los datos originales si el checkbox está marcado
if show_original_data:
    st.subheader("Datos originales")
    st.dataframe(load_data())

    # Crear un histograma de los empleados agrupados por edad
st.subheader("Histograma de empleados por edad")
st.bar_chart(data["Age"].value_counts())

# Crear una gráfica de frecuencias para las unidades funcionales (Unit)
st.subheader("Gráfica de frecuencias para las unidades funcionales")
st.bar_chart(data["Unit"].value_counts())

# Calculate the attrition rate for each city
city_attrition_rate = (
    data.groupby("Hometown")
    .agg(attrition_rate=("Attrition", "mean"))
    .reset_index()
    .sort_values(by="attrition_rate", ascending=False)
)

# Create a bar chart of the cities with the highest attrition rate
st.subheader("Ciudades con mayor tasa de abandono")
fig = px.bar(city_attrition_rate.head(10), x="Hometown", y="attrition_rate",
             title="Ciudades con mayor tasa de abandono",
             labels={"Hometown": "Ciudad", "attrition_rate": "Tasa de abandono"},
             color_discrete_sequence=px.colors.qualitative.Pastel1)
st.plotly_chart(fig)

Overwriting app.py


In [3]:
!streamlit run /content/app.py &>/content/streamlit.log &

In [4]:
!curl -s http://localhost:4040/api/tunnels | python3 -c 'import sys, json; print("Execute the next cell and then go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and then go to the following URL: https://24ba-34-133-245-86.ngrok-free.app
